In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
spark = SparkSession.builder.appName(
    "Ch12 - recipes ML model - are you a dessert? ML PIPELINE"
).config("spark.driver.memory", "8g").getOrCreate()

In [ ]:
food = spark.read.csv("/opt/spark/data/recipes/epi_r.csv", inferSchema=True, header=True)

In [ ]:
print(food.count(), len(food.columns))

In [ ]:
food.printSchema()

In [ ]:
def sanitize_column_name(name):
    answer = name

    for i, j in ((" ", "_"), ("-", "_"), ("/", "_"), ("&", "and")):
        answer = answer.replace(i, j)

    return "".join([char for char in answer if char.isalpha() or char.isdigit() or char == "_"])

In [ ]:
food = food.toDF(*[sanitize_column_name(name) for name in food.columns])

In [ ]:
interesting_columns = ["clove"]

for x in food.columns:
    if x in interesting_columns:
        food.select(x).summary().show()

In [ ]:
import pandas as pd
pd.set_option("display.max.rows", 20)

In [ ]:
is_binary = food.agg(
    *[(F.size(F.collect_set(x)) == 2).alias(x) for x in food.columns]
).toPandas()

is_binary.unstack()

In [ ]:
food.agg(*[F.collect_set(x) for x in ("cakeweek", "wasteless")]).show(
    1, False
)

In [ ]:
food.where("cakeweek > 1.0 or wasteless > 1.0").select(
    "title", "rating", "wasteless", "cakeweek", food.columns[-1]
).show()

In [ ]:
food = food.where(
    (
        F.col("cakeweek").isin([0.0, 1.0]) | F.col("cakeweek").isNull()
    ) &
    (
        F.col("wasteless").isin([0.0, 1.0]) | F.col("wasteless").isNull()
    )
)

print(food.count(), len(food.columns))

In [ ]:
IDENTIFIERS = ["title"]
CONTINUOUS_COLUMNS = [
    "rating",
    "calories",
    "protein",
    "fat",
    "sodium",
]

TARGET_COLUMN = ["dessert"]

BINARY_COLUMNS = [x for x in food.columns if x not in [*IDENTIFIERS, *CONTINUOUS_COLUMNS, *TARGET_COLUMN]]

In [ ]:
food = food.dropna(how="all", subset=[x for x in food.columns if x not in IDENTIFIERS])

food = food.dropna(subset=TARGET_COLUMN)

print(food.count(), len(food.columns))

In [ ]:
food = food.fillna(0.0, subset=BINARY_COLUMNS)

food.where(F.col(BINARY_COLUMNS[0]).isNull()).count() # => 0

In [ ]:
from typing import Optional

@F.udf(T.BooleanType())
def is_a_number(value: Optional[str]) -> bool:
    if not value:
        return True

    try:
        _ = float(value)
    except ValueError:
        return False
    return True

food.where(~is_a_number(F.col("rating"))).select(
    *CONTINUOUS_COLUMNS
).show()


In [ ]:
for column in ["rating", "calories"]:
    food = food.where(is_a_number(F.col(column)))
    food = food.withColumn(column, F.col(column).cast(T.DoubleType()))

print(food.count(), len(food.columns))

In [ ]:
food.select(*CONTINUOUS_COLUMNS).summary(
    "mean",
    "stddev",
    "min",
    "1%",
    "5%",
    "50%",
    "95%",
    "99%",
    "max",
).show()

In [ ]:
inst_sum_of_binary_columns = [
    F.sum(F.col(x)).alias(x) for x in BINARY_COLUMNS
]

sum_of_binary_columns = food.select(*inst_sum_of_binary_columns).head().asDict()

num_rows = food.count()

too_rare_features = [
    k for k, v in sum_of_binary_columns.items() if v < 10 or v > (num_rows - 10)
]

print(len(too_rare_features))

print(too_rare_features)

BINARY_COLUMNS = list(set(BINARY_COLUMNS) - set(too_rare_features))

In [ ]:
food = food.withColumn("protein_ratio", F.col("protein") * 4 / F.col("calories")).withColumn(
    "fat_ratio", F.col("fat") * 9 / F.col("calories")
)

food = food.fillna(0.0, subset=["protein_ratio", "fat_ratio"])

In [ ]:
import pyspark.ml.feature as MF
from pyspark.ml import Pipeline

In [ ]:
OLD_COLS = ["calories", "protein", "fat", "sodium"]
NEW_COLS = ["calories_i", "protein_i", "fat_i", "sodium_i"]

CONTINUOUS_NB = ["rating", "calories_i", "protein_i", "fat_i", "sodium_i"]

# taking care of filling null values with the mean value
imputer = MF.Imputer(strategy="mean", inputCols=OLD_COLS, outputCols=NEW_COLS)

continuous_assembler = MF.VectorAssembler(inputCols=CONTINUOUS_NB, outputCol="continuous")

# taking care to scaling the features to values between 0 and 1
continuous_scaler = MF.MinMaxScaler(inputCol="continuous", outputCol="continuous_scaled")

food_pipeline = Pipeline(stages=[imputer, continuous_assembler, continuous_scaler])

In [ ]:
pre_ml_assembler = MF.VectorAssembler(
    inputCols=BINARY_COLUMNS + ["continuous_scaled", "protein_ratio", "fat_ratio"], outputCol="features"
)

food_pipeline.setStages([imputer, continuous_assembler, continuous_scaler, pre_ml_assembler])

food_pipeline_model = food_pipeline.fit(food)
food_features = food_pipeline_model.transform(food)

food_features.select("title", "dessert", "features").show(5, truncate=30)

In [ ]:
print(food_features.schema["features"])

print(food_features.schema["features"].metadata)

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="dessert", predictionCol="prediction")

food_pipeline.setStages([imputer, continuous_assembler, continuous_scaler, pre_ml_assembler, lr])

train, test = food.randomSplit([0.7, 0.3], 13) # 13 is the seed
train.cache()

food_pipeline_model = food_pipeline.fit(train)
results = food_pipeline_model.transform(test)

In [ ]:
spark.stop()